### Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

### Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

### Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [1]:
import pandas as pd
import numpy as np

### Задание 1. Загрузка данных

In [2]:
horse = pd.read_csv('horse_data.csv', na_values = ['?', np.nan])
sick_horse = horse.iloc[:, [0, 1, 3, 4, 5, 6, 10, 22]]
sick_horse.columns = ['surgery','age','rectal temperature','pulse','respiratory rate','temperature of extremities','pain','outcome']
sick_horse = sick_horse.astype({'rectal temperature':'float64'})
sick_horse['age'].replace(9, 2, inplace = True)
sick_horse.info()
sick_horse

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     298 non-null    float64
 1   age                         299 non-null    int64  
 2   rectal temperature          239 non-null    float64
 3   pulse                       275 non-null    float64
 4   respiratory rate            241 non-null    float64
 5   temperature of extremities  243 non-null    float64
 6   pain                        244 non-null    float64
 7   outcome                     298 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [3]:
sick_horse.describe()

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,298.000000,299.000000,239.000000,275.000000,241.000000,243.000000,244.000000,298.000000
mean,1.395973,1.080268,38.166527,71.934545,30.427386,2.345679,2.942623,1.550336
std,0.489881,0.272162,0.733508,28.680522,17.678256,1.046369,1.303993,0.737967
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


In [4]:
sick_horse.mode()

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,38.0,48.0,20.0,3.0,3.0,1.0


In [ ]:
# данные имеют много пропусков, в столбцах 'pulse' и 'respiratory rate' высокое СКО

### Задание 2. Первичное изучение данных

In [5]:
# поиск межквартильного размаха по пульсу
q1 = sick_horse['pulse'].quantile(0.25)
q3 = sick_horse['pulse'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - (1.5 * IQR) 
upper_bound = q3 + (1.5 * IQR)
remove_outliers = sick_horse[sick_horse['pulse'].between(lower_bound, upper_bound, inclusive=True)]
# вывод выбросов
pd.concat([sick_horse, remove_outliers]).drop_duplicates(keep=False).sort_values('pulse')
# в норме 30 - 40 ударов в минуту, у этих лошадей пульс значительно превышен, возможно они больны

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
40,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
274,1.0,2,38.8,150.0,50.0,1.0,5.0,2.0
54,1.0,2,38.6,160.0,20.0,3.0,3.0,2.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
254,1.0,2,38.8,184.0,84.0,1.0,4.0,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
51,2.0,1,NaN,NaN,NaN,1.0,1.0,1.0
57,1.0,1,NaN,NaN,20.0,4.0,5.0,2.0
73,1.0,2,NaN,NaN,NaN,NaN,NaN,2.0
92,2.0,1,NaN,NaN,NaN,3.0,5.0,2.0


In [6]:
# поиск межквартильного размаха по частоте дыхания
q1 = sick_horse['respiratory rate'].quantile(0.25)
q3 = sick_horse['respiratory rate'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - (1.5 * IQR) 
upper_bound = q3 + (1.5 * IQR)
remove_outliers = sick_horse[sick_horse['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)]
# вывод выбросов
pd.concat([sick_horse, remove_outliers]).drop_duplicates(keep=False).sort_values('respiratory rate')
# в норме 8-16 вдохов и выдохов в минуту, лошадкам явно нехорошо 

# оставляю выбросы и продолжу работу с ними т.к. судя по пульсу, частоте дыхания - это естественные выбросы(лошади болееют)

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
119,1.0,1,39.4,54.0,66.0,1.0,2.0,1.0
102,1.0,2,38.0,140.0,68.0,1.0,3.0,1.0
124,1.0,1,38.0,42.0,68.0,4.0,3.0,1.0
264,2.0,1,NaN,56.0,68.0,3.0,3.0,3.0
228,1.0,2,38.5,120.0,70.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
255,1.0,1,37.5,72.0,NaN,2.0,2.0,3.0
270,1.0,1,37.7,80.0,NaN,3.0,5.0,1.0
273,1.0,1,NaN,76.0,NaN,NaN,NaN,3.0
280,2.0,1,40.0,78.0,NaN,3.0,2.0,2.0


### Задание 3. Работа с пропусками

In [7]:
# данные имеют большой разброс, заполняем пропуски медианнным значением в числовых столбцах
sick_horse['pulse'].fillna(sick_horse['pulse'].median(), inplace=True)

# в категориальных столбцах заполняем модой, пропусков не много
sick_horse['temperature of extremities'].fillna(sick_horse['temperature of extremities'].mode()[0], inplace=True)
sick_horse['surgery'].fillna(sick_horse['surgery'].mode()[0], inplace=True)
sick_horse['outcome'].fillna(sick_horse['outcome'].mode()[0], inplace=True)

# пропусков в этих столбцах > 50, возможно, оценка была затруднена, потому что лошади не давали себя осмотреть
sick_horse['pain'].fillna(sick_horse['pain'].mode()[0], inplace=True)
sick_horse['respiratory rate'].fillna(sick_horse['respiratory rate'].median(), inplace=True)
sick_horse['rectal temperature'].fillna(sick_horse['rectal temperature'].median(), inplace=True)
sick_horse

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,3.0,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,3.0,3.0,2.0
4,2.0,1,38.2,64.0,24.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [8]:
sick_horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     299 non-null    float64
 1   age                         299 non-null    int64  
 2   rectal temperature          299 non-null    float64
 3   pulse                       299 non-null    float64
 4   respiratory rate            299 non-null    float64
 5   temperature of extremities  299 non-null    float64
 6   pain                        299 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB
